In [ ]:
import pandas as pd
import numpy as np
from pandas import Series,DataFrame
df = pd.read_csv('C:/Users/xlong/machinelearning/train.csv')
df = df.drop(['Name', 'Ticket', 'Cabin'], axis=1)
age_mean = df['Age'].mean()
df['Age'] = df['Age'].fillna(age_mean)
from scipy.stats import mode
new_Embarked = df['Embarked'] #这个地方要注意 求众数的时候 NaN会报错,所以要除去NaN 然后在求众数
mode_Embarked = mode(new_Embarked.dropna())#得到的是一个数组 属于moderesult类型
print(mode_Embarked[0][0]) #得到众数
df['Embarked'] = df['Embarked'].fillna(mode_Embarked[0][0])
df['Gender'] = df['Sex'].map({'female': 0, 'male': 1}).astype(int)
pd.get_dummies(df['Embarked'], prefix='Embarked').head(10)
df = pd.concat([df, pd.get_dummies(df['Embarked'], prefix='Embarked')], axis=1)
df = df.drop(['Sex', 'Embarked'], axis=1)
cols = df.columns.tolist()
cols = [cols[1]] + cols[0:1] + cols[2:]
df = df[cols]
train_data = df.values
from sklearn.ensemble import RandomForestClassifier
from sklearn.grid_search import GridSearchCV
parameter_grid = {
    'max_features': [0.5, 1],
    'max_depth': [5, None]
}
grid_search = GridSearchCV(RandomForestClassifier(n_estimators = 100), parameter_grid,
                            cv=5, verbose=3)
grid_search.fit(train_data[0:,2:], train_data[0:,0])
grid_search.grid_scores_
sorted(grid_search.grid_scores_, key=lambda x: x.mean_validation_score)
grid_search.best_score_
grid_search.best_params_
model = RandomForestClassifier(n_estimators = 100, max_features=0.5, max_depth=5.0)
model = model.fit(train_data[0:,2:],train_data[0:,0])
df_test = pd.read_csv('C:/Users/xlong/machinelearning/test.csv')
df_test = df_test.drop(['Name', 'Ticket', 'Cabin'], axis=1
df_test['Age'] = df_test['Age'].fillna(age_mean)
fare_means = df.pivot_table('Fare', index='Pclass')
fill_Fare = df_test['Pclass'][np.where(np.isnan(df_test['Fare'].values))[0][0]]#利用np.where找出nan的索引 求出Pclass的值 所对应的平均值
df_test['Fare'] = df_test['Fare'].fillna(fare_means['Fare'][fill_Fare])
df_test['Gender'] = df_test['Sex'].map({'female': 0, 'male': 1}).astype(int)
df_test = pd.concat([df_test, pd.get_dummies(df_test['Embarked'], prefix='Embarked')],
                axis=1)
df_test = df_test.drop(['Sex', 'Embarked'], axis=1)
test_data = df_test.values
output = model.predict(test_data[:,1:])
result = np.c_[test_data[:,0].astype(int), output.astype(int)]
df_result = pd.DataFrame(result[:,0:2], columns=['PassengerId', 'Survived'])
df_result.to_csv('C:/Users/xlong/machinelearning/titanic_1-3.csv', index=False)
